In [22]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pandas as pd


from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer = PorterStemmer()

from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
tfidf = TfidfVectorizer(
    min_df = 1,
    max_df = 0.95,
    max_features = 8000,
    stop_words = 'english'
)

import re
import sys
sys.path.append(r"../Desktop/performance_team_bidlist_code/")
import APIs.Brandcdn_Library as blib

stop = stopwords.words('english')
stop.append('best')
stop.append('near')
stop.append('close')
stop.append('nearest')
stop.append('closest')
STOP_WORDS = STOP_WORDS | set(stop)

# To PHP Translation

# Packages without Translation
### 1. Nltk (synonyms and stopwords) --> stopwords can be saved but synonyms more tricky
### 2. texthero for text cleaning
### 3. TFIDF (vectorizer, dense-matrix, normalization)

In [3]:
"""
word_dict: 
    Dictionary of how many time a word occurs in the keyword lsit
"""
def word_dict(df,column_name):
    words2 = df[column_name]
    d_word2 = {}
    for val in words2:
       for wrd in val.split(' '):
        d_word2[wrd] = d_word2.get(wrd,0)+1
    return d_word2

"""
stem_dict: Dictionary of words that stem to the same value
    If they stem to the same value transform the original word into the most used word
    Example: Doctors occurs 20 times, Doctor occurs 30 times in the keywords list
             Doctors -> Doctor 
             Doctor -> Doctor
"""
def stem_dict(d):
    d_stem = {}
    for word in d:
        d_stem[stemmer.stem(word)] = []
    for word in d:
        d_stem[stemmer.stem(word)].append(word)
    d1 = {}
    for val in d_stem:
        if(len(d_stem[val])>1):
            d1[min(d_stem[val],key = len)] = d_stem[val]
    d_out = {}        
    for val in d1:
        for word in d1[val]:
            if val!=word:
                d_out[word] = val
    return d_out
"""
syn_dict: Dictionary of words that has a synonym that occuring in the keywords
   Example: job, work (if work occurs more frequently)
           job -> work
           work -> work
"""
def syn_dict(d):

    d_syn = {}
    for word in d:

        synonyms = []

        for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                synonyms.append(l.name())
        l1=[]
        for word2 in set(synonyms):
            if word2 in d:
                l1.append(word2)
        if(len(l1)>1):
            d_syn[word] = l1
    d_rep = {}
    for val in d_syn:
        max_value = max([d[w] for w in d_syn[val]])
        if(val!=d_syn[val][[d[w] for w in d_syn[val]].index(max_value)]):
            if(val!='good' and val !='best' and d_syn[val][[d[w] for w in d_syn[val]].index(max_value)]!='good'):
                d_rep[val] = d_syn[val][[d[w] for w in d_syn[val]].index(max_value)]
    return d_rep
"""
get_top_keywords: returns the top 6-8 keywords for an adgroup once the clustering has already been done
"""
def get_top_keywords(data, clusters, labels, n_terms):
    df = pd.DataFrame(data.todense()).groupby(clusters).mean()
    d = {}
    d1 = {}
    out = []
    for i,r in df.iterrows():
        out.append([labels[t] for t in np.argsort(r)[-n_terms:]])
    for i in range(len(out)):
        d[i] = str(out[i][::-1])
        d1[i] = out[i][::-1]
        
    return d,d1    
def advertiser_adgroup(data):
    # .str.contains("substring")
    return 0

nlp = spacy.load('en_core_web_sm')

def clean_text(text):
    doc = nlp(text)
    cleaned_text = " ".join(token.lemma_ for token in doc if not token.is_stop 
                            and not token.is_punct 
                            and not token.is_digit)
    return cleaned_text.lower()

In [4]:
# unique_display_id: only value that needs to be input into the program. 
inputs = [16329538734251683884] 

# This query returns the keyword list for each io in inputs

In [5]:
query2 = """
SELECT mss.id
    , mss.unique_display_id
    , subp.id as submitted_prod_id
    , subp.campaign_name
    , ikd.advertiser_name
    , ikd.search_terms as keyword
FROM mpq_sessions_and_submissions mss
JOIN cp_submitted_products subp ON subp.mpq_id = mss.id
JOIN cp_io_keywords_data ikd ON ikd.cp_submitted_products_id = subp.id
WHERE mss.unique_display_id IN ( """
for i,val in enumerate(inputs):
    if(i!=len(inputs)-1):
        query2+=str(val)+str(',') 
    else:
        query2+=str(val) + ')'
query2

'\nSELECT mss.id\n    , mss.unique_display_id\n    , subp.id as submitted_prod_id\n    , subp.campaign_name\n    , ikd.advertiser_name\n    , ikd.search_terms as keyword\nFROM mpq_sessions_and_submissions mss\nJOIN cp_submitted_products subp ON subp.mpq_id = mss.id\nJOIN cp_io_keywords_data ikd ON ikd.cp_submitted_products_id = subp.id\nWHERE mss.unique_display_id IN ( 16329538734251683884)'

In [6]:
df2 =  blib.bcdn_select_query(query2)

C:\Users\brent.glassman\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
# this line removes any null keyword lists
df2 = df2[df2['keyword']!='[]'].reset_index(drop = True)
df2

,id,unique_display_id,submitted_prod_id,campaign_name,advertiser_name,keyword
0,660172,16329538734251683884,19548617,DMA MEMPHIS-SEM,Stern Cardiovascular,"[""stern cardiovascular"",""stern cardiovascular..."


# Steps
1. TextHero
    a. remove empty strings
    b. make keywords all lowercase
    c. remove numbers
    d. remove punctuation
    e. remove any accents on letters (diacritics)
    f. remove whitespace
    g. remove stopwords (ammended from prior experience)
2. Determine adgroups by number of keywords (more keywords more adgroups)
3. Replace similar words like doctors -> doctor 
4. Replace synonyms like work -> job
5. Run Tfidf (Determines how important words are)
6. Run Kmeans (Cluster similar keywords together)
7. analyze each cluster (adgroup) for the most relevant words
8. Flag any irrelevant hacky words like (pizza, amazon, walmart)
9. Take keywords with advertiser name in them and add them to the advertiser ad group
10. Output File with campaign & advertiser name

In [10]:
for val in df2['submitted_prod_id']:
    df = df2[df2['submitted_prod_id'] == val].copy()
    df['keyword'] = df.keyword.apply(lambda x: x[1:-1].split(','))
    df = df.explode('keyword')
    df['keyword'] = df['keyword'].apply(lambda x: x.replace('"', ''))
    df = df[df['keyword']!=''].reset_index(drop = True)
    df = df.drop_duplicates()
    df['clean'] = df['keyword'].apply(clean_text)
    #df['clean'] = hero.clean(df['keyword'])
    #df['clean'] = hero.remove_stopwords(df['clean'],stop)
    
    Nclust = max(min(round(df['clean'].nunique()/42),8),2)
    d_word2 = word_dict(df,'clean')
    
    d_stem = stem_dict(d_word2)
    d_stem = {rf'\b{k}\b': v for k, v in d_stem.items()}
    df['test2'] = df['clean'].replace(d_stem,regex = True)
    
    d_word3 = word_dict(df,'test2')
    d_rep = syn_dict(d_word3)
    d_rep = {rf'\b{k}\b': v for k, v in d_rep.items()}
    df['test3'] = df['test2'].replace(d_rep,regex = True)
    
    tfidf.fit(df.test3)
    text3 = tfidf.transform(df.test3)    
    
    km = KMeans(n_clusters=Nclust,  random_state=20)
    clusters2 = km.fit_predict(preprocessing.normalize(text3))
    df['label4'] = clusters2
    
    overview1,overview2 = get_top_keywords(text3,clusters2,tfidf.get_feature_names(), max(Nclust,6))
    
    df['Cluster_Overview'] = df['label4']
    df = df.replace({'Cluster_Overview': overview1})
    
    df_out = df[['unique_display_id','campaign_name','advertiser_name','keyword','label4','Cluster_Overview']].copy()


    hacks = ['walmart','amazon','pizza']
    df_out['flag']=np.where(~df_out['keyword'].str.contains('|'.join(hacks), case=False),0,1)
    df_out[df_out['flag']==1]
    
    # str1 =  hero.clean(df_out['campaign_name']).unique()[0]
    # str2 =  hero.clean(df_out['advertiser_name']).unique()[0]
    str1 =  df_out['campaign_name'].unique()[0]
    str2 =  df_out['advertiser_name'].unique()[0]
    str3 = df_out['unique_display_id'].unique()[0]
    
    df_out['label4']=np.where(~hero.clean(df_out['keyword']).str.contains(str2, case=False),df_out['label4'],'Advertiser Ad Group')

    
    
    #str_out = 'Grouping_Outputs/keyword_grouping__' + str1 +'__'+ str2 +'__'+ str3 + '.csv' 
    str_out = 'Keyword_Grouping/Grouping_Outputs/keyword_grouping__' + str1 +'__'+ str2 +'__'+ str3 + '.csv' 
    df_out.rename(columns = {'id': 'Original_adgroup','label4':'Ad Group #', 'Cluster_Overview': 'Ad Group Overview'}).sort_values(by='Ad Group #').to_csv(str_out)


C:\Users\brent.glassman\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [56]:
print(df_out['label4'].unique())
df_out[df_out['label4']=='Advertiser Ad Group']

['Advertiser Ad Group' '0' '1' '2' '3']


,unique_display_id,campaign_name,advertiser_name,keyword,label4,Cluster_Overview,flag
0,16329538734251683884,DMA MEMPHIS-SEM,Stern Cardiovascular,stern cardiovascular,Advertiser Ad Group,"['stern', 'cardiovascular', 'southaven', 'clin...",0
1,16329538734251683884,DMA MEMPHIS-SEM,Stern Cardiovascular,stern cardiovascular,Advertiser Ad Group,"['stern', 'cardiovascular', 'southaven', 'clin...",0
8,16329538734251683884,DMA MEMPHIS-SEM,Stern Cardiovascular,stern cardiovascular foundation,Advertiser Ad Group,"['stern', 'cardiovascular', 'southaven', 'clin...",0
14,16329538734251683884,DMA MEMPHIS-SEM,Stern Cardiovascular,stern cardiovascular southaven ms,Advertiser Ad Group,"['stern', 'cardiovascular', 'southaven', 'clin...",0
23,16329538734251683884,DMA MEMPHIS-SEM,Stern Cardiovascular,stern cardiovascular munford tn,Advertiser Ad Group,"['stern', 'cardiovascular', 'southaven', 'clin...",0
28,16329538734251683884,DMA MEMPHIS-SEM,Stern Cardiovascular,stern cardiovascular germantown tn,Advertiser Ad Group,"['stern', 'cardiovascular', 'southaven', 'clin...",0
32,16329538734251683884,DMA MEMPHIS-SEM,Stern Cardiovascular,stern cardiovascular memphis tn,Advertiser Ad Group,"['stern', 'cardiovascular', 'southaven', 'clin...",0
34,16329538734251683884,DMA MEMPHIS-SEM,Stern Cardiovascular,stern cardiovascular memphis,Advertiser Ad Group,"['stern', 'cardiovascular', 'southaven', 'clin...",0
35,16329538734251683884,DMA MEMPHIS-SEM,Stern Cardiovascular,stern cardiovascular oxford ms,Advertiser Ad Group,"['stern', 'cardiovascular', 'southaven', 'clin...",0
36,16329538734251683884,DMA MEMPHIS-SEM,Stern Cardiovascular,stern cardiovascular southaven mississippi,Advertiser Ad Group,"['stern', 'cardiovascular', 'southaven', 'clin...",0


In [13]:
df[['keyword','clean']][0:50]

,keyword,clean
0,stern cardiovascular,stern cardiovascular
1,stern cardiovascular,stern cardiovascular
2,heart attack,heart attack
3,sutherland cardiology,sutherland cardiology
4,chest pain,chest pain
5,atrial fibrillation,atrial fibrillation
6,congestion heart failure,congestion heart failure
7,congestive heart failure,congestive heart failure
8,stern cardiovascular foundation,stern cardiovascular foundation
9,varicose veins,varicose vein
